# Prediccion de imagenes MNIST Fashion

In [2]:
from google.colab import drive
drive.mount('/content/drive')

DRIVE_PATH = '/content/drive/MyDrive/datasets/fashion_mnist/'
ZIP_NUEVOS = 'fashion_mnist_nuevos.zip'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# ==========================================================
# NUEVA NOTEBOOK — CELDA ÚNICA: Cargar resultados.zip y predecir un ZIP nuevo
# Entradas (subidas manualmente a /content):
#   - /content/resultados.zip   (bundle del modelo + metadata)
#   - /content/nuevo.zip        (nuevo dataset para inferencia)
# Salidas:
#   - /content/predicciones.csv
# ==========================================================
import os, glob, json, zipfile, shutil, time
import numpy as np
import tensorflow as tf

# -------------------------
# 0) RUTAS
# -------------------------
RESULTS_ZIP = "/content/resultados_fashion_mnist.zip"
NEW_ZIP     = os.path.join(DRIVE_PATH, ZIP_NUEVOS)  # cambia el nombre si llega con otro nombre

RESULTS_DIR = "/content/resultados"
NEW_WORKDIR = "/content/new_data"

EXTS = (".jpg",".jpeg",".png",".bmp",".webp")
AUTOTUNE = tf.data.AUTOTUNE

assert os.path.isfile(RESULTS_ZIP), "Falta /content/resultados.zip (súbelo manualmente)"
assert os.path.isfile(NEW_ZIP),     f"Falta /content/{ZIP_NUEVOS}"

# -------------------------
# 1) DESCOMPRIMIR resultados.zip
# -------------------------
if os.path.isdir(RESULTS_DIR):
    shutil.rmtree(RESULTS_DIR)
os.makedirs(RESULTS_DIR, exist_ok=True)

with zipfile.ZipFile(RESULTS_ZIP, "r") as z:
    z.extractall(RESULTS_DIR)

print("✅ resultados.zip extraído en:", RESULTS_DIR)
print("Contenido:", sorted(os.listdir(RESULTS_DIR))[:20])

MODEL_PATH = os.path.join(RESULTS_DIR, "model.keras")
META_PATH  = os.path.join(RESULTS_DIR, "metadata.json")

assert os.path.isfile(MODEL_PATH), f"No existe {MODEL_PATH}"
assert os.path.isfile(META_PATH),  f"No existe {META_PATH}"

# -------------------------
# 2) CARGAR METADATA + MODELO
# -------------------------
with open(META_PATH, "r", encoding="utf-8") as f:
    meta = json.load(f)

IMG_SIZE  = tuple(meta["img_size"])
CHANNELS  = int(meta["channels"])
CLASSES   = list(meta["classes"])
BATCH     = int(meta.get("batch_final", 32))

print("\nCONFIG INFERENCIA:")
print("  IMG_SIZE :", IMG_SIZE)
print("  CHANNELS :", CHANNELS)
print("  BATCH    :", BATCH)
print("  #CLASSES :", len(CLASSES))

model = tf.keras.models.load_model(MODEL_PATH)
print("\n✅ Modelo cargado:", MODEL_PATH)

# -------------------------
# 3) DESCOMPRIMIR NUEVO ZIP
# -------------------------
if os.path.isdir(NEW_WORKDIR):
    shutil.rmtree(NEW_WORKDIR)
os.makedirs(NEW_WORKDIR, exist_ok=True)

with zipfile.ZipFile(NEW_ZIP, "r") as z:
    z.extractall(NEW_WORKDIR)

print("\n✅ nuevo.zip extraído en:", NEW_WORKDIR)

# -------------------------
# 4) ENCONTRAR TODAS LAS IMÁGENES (sin asumir carpetas por clase)
#    (Soporta: imágenes en raíz, o en subcarpetas a cualquier profundidad)
# -------------------------
def list_all_images_recursive(root_dir, exts=EXTS):
    files = []
    for r, _, fs in os.walk(root_dir):
        for fn in fs:
            if fn.lower().endswith(exts):
                files.append(os.path.join(r, fn))
    return sorted(files)

files = list_all_images_recursive(NEW_WORKDIR)
if len(files) == 0:
    raise ValueError(f"No encontré imágenes dentro de {NEW_ZIP}")

print("Imágenes encontradas:", len(files))
print("Ejemplo:", files[0])

# -------------------------
# 5) PIPELINE TF.DATA (mismo preprocesamiento que entrenaste)
# -------------------------
def decode_image(path, img_size, channels):
    img = tf.io.read_file(path)
    img = tf.io.decode_image(img, channels=channels, expand_animations=False)
    img = tf.image.resize(img, img_size, antialias=True)
    img = tf.cast(img, tf.float32) / 255.0
    return img

def make_ds(paths, batch):
    ds = tf.data.Dataset.from_tensor_slices(paths)
    ds = ds.map(lambda p: decode_image(p, IMG_SIZE, CHANNELS), num_parallel_calls=AUTOTUNE)
    ds = ds.batch(batch).prefetch(AUTOTUNE)
    return ds

ds = make_ds(files, BATCH)

# -------------------------
# 6) PREDICCIÓN + CSV
# -------------------------
probs = model.predict(ds, verbose=0)
pred_idx = np.argmax(probs, axis=1)
pred_cls = [CLASSES[i] for i in pred_idx]
conf = np.max(probs, axis=1)

out_csv = "/content/predicciones.csv"
import csv
with open(out_csv, "w", newline="", encoding="utf-8") as f:
    w = csv.writer(f)
    w.writerow(["filepath", "pred_idx", "pred_class", "confidence"])
    for p, i, c, cf in zip(files, pred_idx, pred_cls, conf):
        w.writerow([p, int(i), c, float(cf)])

print("\n✅ Listo. CSV:", out_csv)
print("Primeras 5 predicciones:")
for k in range(min(5, len(files))):
    print(f"  {os.path.basename(files[k])} -> {pred_cls[k]} (conf={conf[k]:.3f})")


✅ resultados.zip extraído en: /content/resultados
Contenido: ['README_INFERENCIA.txt', 'infer_from_zip.py', 'metadata.json', 'model.keras', 'weights.best.keras']

CONFIG INFERENCIA:
  IMG_SIZE : (32, 32)
  CHANNELS : 1
  BATCH    : 256
  #CLASSES : 10


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 34 variables whereas the saved optimizer has 38 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))



✅ Modelo cargado: /content/resultados/model.keras

✅ nuevo.zip extraído en: /content/new_data
Imágenes encontradas: 50
Ejemplo: /content/new_data/0/0_1.png

✅ Listo. CSV: /content/predicciones.csv
Primeras 5 predicciones:
  0_1.png -> 0 (conf=0.951)
  0_2.png -> 0 (conf=0.989)
  0_3.png -> 0 (conf=0.988)
  0_4.png -> 0 (conf=0.996)
  0_5.png -> 0 (conf=0.665)


In [24]:
import pandas as pd

predicciones = pd.read_csv('predicciones.csv')
predicciones.shape

(50, 4)

In [25]:
predicciones['pred_class'] = predicciones['pred_class'].astype(str)
predicciones['clase'] = predicciones['filepath'].str.replace('/content/new_data/','').str[:-8]
predicciones.drop(columns=['filepath'], inplace=True)

In [26]:
predicciones['pred_class'].describe()

,pred_class
count,50
unique,10
top,0
freq,6


In [27]:
correct_pred = lambda x: x['pred_class'] in x['clase']
predicciones['prediccion acertada'] = predicciones.apply(correct_pred, axis=1)


predicciones.groupby(by = 'clase', as_index=False).agg({'prediccion acertada': 'sum',
                                                      'confidence': ['min', 'mean', 'median', 'max']})

clase prediccion acertada confidence                              
                        sum        min      mean    median       max
0     0                   5   0.665062  0.917834  0.988371  0.995951
1     1                   5   0.999976  0.999986  0.999988  0.999996
2     2                   4   0.501632  0.791058  0.804159  0.942174
3     3                   5   0.781484  0.945576  0.986138  0.988180
4     4                   4   0.572562  0.813737  0.971770  0.974520
5     5                   5   0.988765  0.997300  0.999858  0.999949
6     6                   4   0.439273  0.750493  0.789785  0.978938
7     7                   5   0.988345  0.996774  0.998727  0.999901
8     8                   5   0.999282  0.999742  0.999952  0.999992
9     9                   5   0.959441  0.988601  0.995313  0.997199